### Para cada caso listado no ticket #242:
1. Procurar todos os casos no ES em que a condição acontece
+ Para cada caso do item 1, consertar o valor encontrado pelo esperado no CouchDB
+ Reindexar banco (fora deste script)

In [50]:
import couchdb
import elasticsearch
import json

In [235]:
# Set ES connection
es = elasticsearch.Elasticsearch('http://10.10.100.49/')
es.ping()

True

In [236]:
# Set CouchDB connection
couch = couchdb.Server('http://10.10.100.49/couchdb')

In [77]:
def search_es(body):
    docs = es.search(index='_all', body=body, doc_type='occurrence', size=5000)
    print "Got %d documents" % len(docs['hits']['hits'])
    return docs

In [362]:
def print_analysis(begin_space, end_space, mistyped, normal, limbo):
    print "Foram encontrados %d registros com espaço no inicio." % len(begin_space)
    print "Foram encontrados %d registros com espaço no final." % len(end_space)
    print "Foram encontrados %d registros escritos errados." % len(mistyped)
    print "Foram encontrados %d registros normais." % len(normal)
    print "Foram encontrados %d registros no limbo." % len(limbo)

def get_ids_to_fix(docs, mistyped_search, correct):
    begin_space = []
    end_space = []
    mistyped = []
    normal = []
    limbo = []
    id_test = ''
    for idx, doc in enumerate(docs['hits']['hits']):
        precision = doc['_source']['coordinateUncertaintyInMeters']
        # Check for empty spaces before
        if precision[0].isspace():
            begin_space.append((doc['_index'], doc['_id']))
            if doc['_id'] == id_test:
                print "begin"
        # Check for empty spaces after
        elif precision[len(precision)-1].isspace():
            end_space.append((doc['_index'], doc['_id']))
            if doc['_id'] == id_test:
                print "end"
        # Check for mistyped conditions
        elif precision.lower() in mistyped_search:
            mistyped.append((doc['_index'], doc['_id']))
            if doc['_id'] == id_test:
                print "mistyped"
        elif precision == correct:
            normal.append((doc['_index'], doc['_id']))
            if doc['_id'] == id_test:
                print precision
                print "normal"
        else:
            limbo.append((doc['_index'], doc['_id']))
            if doc['_id'] == id_test:
                print precision
                print "limbo"

        if not (idx % 100):
            print "Analisando doc %i" % idx
    print "\n\n"
    print_analysis(begin_space, end_space, mistyped, normal, limbo)
    return (begin_space, end_space, mistyped, normal, limbo)

def update_couch(to_fix, correct):
    not_found = []
    for doc in to_fix:
        # Get alias or index name
        aliases = es.indices.get_aliases(index=doc[0])
        if len(aliases[doc[0]]["aliases"]) == 0:
            doc_index = doc[0]
        else:
            doc_index = aliases[doc[0]]["aliases"].keys()[0]
        doc_id = doc[1]
        
        print (doc_index, doc_id)
        
        # Get document
        try:
            db = couch[doc_index]
        except couchdb.ServerError:
            print doc_index
            break
        doc_couch = db.get(doc_id)
        if doc_couch == None:
            not_found.append(doc_id)
            continue

        doc_couch['coordinateUncertaintyInMeters'] = correct
        db.update([doc_couch])
    return not_found

#### Pós-análise

In [266]:
from collections import Counter

In [319]:
error = []
total = []
body_id = {
    "query": {"match": {
       "_id": ""
    }},
    "fields": [
       "coordinateUncertaintyInMeters"
    ]
}

for doc in mistyped:
    # Get alias or index name
    aliases = es.indices.get_aliases(index=doc[0])
    if len(aliases[doc[0]]["aliases"]) == 0:
        doc_index = doc[0]
    else:
        doc_index = aliases[doc[0]]["aliases"].keys()[0]
    doc_id = doc[1]
    body_id['query']['match']['_id'] = doc_id
    doc = es.search(index=doc_index, body=json.dumps(body_id), doc_type='occurrence', size=1)
    precision = doc['hits']['hits'][0]['fields']['coordinateUncertaintyInMeters']
    #print precision
    total.append(precision[0])
print Counter(total)

Counter({u'Centroide de reserva/parque': 7, u'centroide de reserva/parque': 2})


#### Caso 1: 0 a 250 m

In [368]:
body="""{
   "query": {
      "bool": {
         "should": [
            {
               "match": {
                  "coordinateUncertaintyInMeters": " 0 a 250 m"
               }
            },
            {
               "match": {
                  "coordinateUncertaintyInMeters": " 0 a 250m"
               }
            },
            {
               "match": {
                  "coordinateUncertaintyInMeters": " 0 a 250"
               }
            }
         ],
         "minimum_number_should_match": 1
      }
   }
}"""
docs = search_es(body)

Got 737 documents


In [369]:
(begin_space, end_space, mistyped, normal, limbo) = get_ids_to_fix(docs, ["0 a 250", "0 a 250m"], "0 a 250 m")
to_fix = begin_space + end_space + mistyped + limbo

Analisando doc 0
Analisando doc 100
Analisando doc 200
Analisando doc 300
Analisando doc 400
Analisando doc 500
Analisando doc 600
Analisando doc 700



Foram encontrados 0 registros com espaço no inicio.
Foram encontrados 0 registros com espaço no final.
Foram encontrados 0 registros escritos errados.
Foram encontrados 737 registros normais.
Foram encontrados 0 registros no limbo.


In [259]:
not_found = update_couch(to_fix, "0 a 250 m")
print "Não foram encontrados %d documentos" % len(not_found)

Não foram encontrados 0 documentos


#### Caso 2: 250 a 1000 m

In [370]:
body="""{
   "query": {
      "bool": {
         "should": [
            {
               "match": {
                  "coordinateUncertaintyInMeters": "250 a 1000 m"
               }
            },
            {
               "match": {
                  "coordinateUncertaintyInMeters": "250 a 1000"
               }
            },
            {
               "match": {
                  "coordinateUncertaintyInMeters": "250 a 1000m"
               }
            },
            {
               "match": {
                  "coordinateUncertaintyInMeters": "251 a 1000 m"
               }
            },
            {
               "match": {
                  "coordinateUncertaintyInMeters": "0,250 a 1km"
               }
            },
            {
               "match": {
                  "coordinateUncertaintyInMeters": "0,251 a 1km"
               }
            },
            {
               "match": {
                  "coordinateUncertaintyInMeters": "205 a 1000 m"
               }
            }
         ],
         "minimum_number_should_match": 1
      }
   }
}"""
docs = search_es(body)

Got 2482 documents


In [371]:
(begin_space, end_space, mistyped, normal, limbo) = get_ids_to_fix(docs, ["250 a 1000", "250 a 1000m", "251 a 1000 m",
                                                                  "0,250 a 1km", "0,251 a 1km", "205 a 1000 m"], "250 a 1000 m")
to_fix = begin_space + end_space + mistyped + limbo

Analisando doc 0
Analisando doc 100
Analisando doc 200
Analisando doc 300
Analisando doc 400
Analisando doc 500
Analisando doc 600
Analisando doc 700
Analisando doc 800
Analisando doc 900
Analisando doc 1000
Analisando doc 1100
Analisando doc 1200
Analisando doc 1300
Analisando doc 1400
Analisando doc 1500
Analisando doc 1600
Analisando doc 1700
Analisando doc 1800
Analisando doc 1900
Analisando doc 2000
Analisando doc 2100
Analisando doc 2200
Analisando doc 2300
Analisando doc 2400



Foram encontrados 0 registros com espaço no inicio.
Foram encontrados 0 registros com espaço no final.
Foram encontrados 0 registros escritos errados.
Foram encontrados 2482 registros normais.
Foram encontrados 0 registros no limbo.


In [345]:
not_found = update_couch(to_fix, "250 a 1000 m")
print "Não foram encontrados %d documentos" % len(not_found)

Não foram encontrados 0 documentos


#### Caso 3: 1 a 5 km

In [372]:
body="""{
   "query": {
      "bool": {
         "should": [
            {
               "match": {
                  "coordinateUncertaintyInMeters": "1 a 5 km"
               }
            },
            {
               "match": {
                  "coordinateUncertaintyInMeters": "1 a 5km"
               }
            }
         ],
         "minimum_number_should_match": 1
      }
   }
}"""
docs = search_es(body)

Got 2670 documents


In [373]:
(begin_space, end_space, mistyped, normal, limbo) = get_ids_to_fix(docs, ["1 a 5km"], "1 a 5 km")
to_fix = begin_space + end_space + mistyped + limbo

Analisando doc 0
Analisando doc 100
Analisando doc 200
Analisando doc 300
Analisando doc 400
Analisando doc 500
Analisando doc 600
Analisando doc 700
Analisando doc 800
Analisando doc 900
Analisando doc 1000
Analisando doc 1100
Analisando doc 1200
Analisando doc 1300
Analisando doc 1400
Analisando doc 1500
Analisando doc 1600
Analisando doc 1700
Analisando doc 1800
Analisando doc 1900
Analisando doc 2000
Analisando doc 2100
Analisando doc 2200
Analisando doc 2300
Analisando doc 2400
Analisando doc 2500
Analisando doc 2600



Foram encontrados 0 registros com espaço no inicio.
Foram encontrados 0 registros com espaço no final.
Foram encontrados 0 registros escritos errados.
Foram encontrados 2670 registros normais.
Foram encontrados 0 registros no limbo.


In [356]:
not_found = update_couch(to_fix, "1 a 5 km")
print "Não foram encontrados %d documentos" % len(not_found)

(u'endemicas_rio_de_janeiro', u'occurrence:rj:857245')
(u'endemicas_rio_de_janeiro', u'occurrence:rj:4570575')
Não foram encontrados 0 documentos


#### Caso 3: 5 a 10 km

In [374]:
body="""{
   "query": {
      "bool": {
         "should": [
            {
               "match": {
                  "coordinateUncertaintyInMeters": "5 a 10 km"
               }
            },
            {
               "match": {
                  "coordinateUncertaintyInMeters": "5 a 10km"
               }
            }
         ],
         "minimum_number_should_match": 1
      }
   }
}"""
docs = search_es(body)

Got 2834 documents


In [375]:
(begin_space, end_space, mistyped, normal, limbo) = get_ids_to_fix(docs, ["5 a 10km"], "5 a 10 km")
to_fix = begin_space + end_space + mistyped + limbo

Analisando doc 0
Analisando doc 100
Analisando doc 200
Analisando doc 300
Analisando doc 400
Analisando doc 500
Analisando doc 600
Analisando doc 700
Analisando doc 800
Analisando doc 900
Analisando doc 1000
Analisando doc 1100
Analisando doc 1200
Analisando doc 1300
Analisando doc 1400
Analisando doc 1500
Analisando doc 1600
Analisando doc 1700
Analisando doc 1800
Analisando doc 1900
Analisando doc 2000
Analisando doc 2100
Analisando doc 2200
Analisando doc 2300
Analisando doc 2400
Analisando doc 2500
Analisando doc 2600
Analisando doc 2700
Analisando doc 2800



Foram encontrados 0 registros com espaço no inicio.
Foram encontrados 0 registros com espaço no final.
Foram encontrados 0 registros escritos errados.
Foram encontrados 2834 registros normais.
Foram encontrados 0 registros no limbo.


In [363]:
not_found = update_couch(to_fix, "5 a 10 km")
print "Não foram encontrados %d documentos" % len(not_found)

(u'endemicas_rio_de_janeiro', u'occurrence:rj:3440948')
(u'endemicas_rio_de_janeiro', u'occurrence:rj:4034738')
Não foram encontrados 0 documentos


#### Caso 4: 10 a 50 km

In [376]:
body="""{
   "query": {
      "bool": {
         "should": [
            {
               "match": {
                  "coordinateUncertaintyInMeters": "10 a 50 km"
               }
            },
            {
               "match": {
                  "coordinateUncertaintyInMeters": "10 a 50km"
               }
            }
         ],
         "minimum_number_should_match": 1
      }
   }
}"""
docs = search_es(body)

Got 263 documents


In [377]:
(begin_space, end_space, mistyped, normal, limbo) = get_ids_to_fix(docs, ["10 a 50km"], "10 a 50 km")
to_fix = begin_space + end_space + mistyped + limbo

Analisando doc 0
Analisando doc 100
Analisando doc 200



Foram encontrados 0 registros com espaço no inicio.
Foram encontrados 0 registros com espaço no final.
Foram encontrados 0 registros escritos errados.
Foram encontrados 263 registros normais.
Foram encontrados 0 registros no limbo.


In [300]:
not_found = update_couch(to_fix, "10 a 50 km")
print "Não foram encontrados %d documentos" % len(not_found)

Não foram encontrados 0 documentos


#### Caso 5: 50 a 100 km

In [378]:
body="""{
   "query": {
      "bool": {
         "should": [
            {
               "match": {
                  "coordinateUncertaintyInMeters": "50 a 100 km"
               }
            }
         ],
         "minimum_number_should_match": 1
      }
   }
}"""
docs = search_es(body)

Got 22 documents


In [379]:
(begin_space, end_space, mistyped, normal, limbo) = get_ids_to_fix(docs, ["50 a 100km"], "50 a 100 km")
to_fix = begin_space + end_space + mistyped + limbo

Analisando doc 0



Foram encontrados 0 registros com espaço no inicio.
Foram encontrados 0 registros com espaço no final.
Foram encontrados 0 registros escritos errados.
Foram encontrados 22 registros normais.
Foram encontrados 0 registros no limbo.


In [304]:
not_found = update_couch(to_fix, "50 a 100 km")
print "Não foram encontrados %d documentos" % len(not_found)

Não foram encontrados 0 documentos


#### Caso 6: centroide de municipio

In [380]:
body="""{
   "query": {
      "bool": {
         "should": [
            {
               "match": {
                  "coordinateUncertaintyInMeters": "centroide de municipio"
               }
            }, 
            {
               "match": {
                  "coordinateUncertaintyInMeters": "centroide de munic"
               }
            }, 
            {
               "match": {
                  "coordinateUncertaintyInMeters": "centroide municipio"
               }
            }
         ],
         "minimum_number_should_match": 1
      }
   }
}"""
docs = search_es(body)

Got 2432 documents


In [381]:
(begin_space, end_space, mistyped, normal, limbo) = get_ids_to_fix(docs, ["centroide de munic", "centroide municipio"], "centroide de municipio")
to_fix = begin_space + end_space + mistyped + limbo

Analisando doc 0
Analisando doc 100
Analisando doc 200
Analisando doc 300
Analisando doc 400
Analisando doc 500
Analisando doc 600
Analisando doc 700
Analisando doc 800
Analisando doc 900
Analisando doc 1000
Analisando doc 1100
Analisando doc 1200
Analisando doc 1300
Analisando doc 1400
Analisando doc 1500
Analisando doc 1600
Analisando doc 1700
Analisando doc 1800
Analisando doc 1900
Analisando doc 2000
Analisando doc 2100
Analisando doc 2200
Analisando doc 2300
Analisando doc 2400



Foram encontrados 0 registros com espaço no inicio.
Foram encontrados 0 registros com espaço no final.
Foram encontrados 0 registros escritos errados.
Foram encontrados 2432 registros normais.
Foram encontrados 0 registros no limbo.


In [313]:
not_found = update_couch(to_fix, "centroide de municipio")
print "Não foram encontrados %d documentos" % len(not_found)

Não foram encontrados 0 documentos


#### Caso 7: centroide de uc

In [382]:
body="""{
   "query": {
      "bool": {
         "should": [
            {
               "match": {
                  "coordinateUncertaintyInMeters": "centroide de uc"
               }
            }, 
            {
               "match": {
                  "coordinateUncertaintyInMeters": "centroide de reserva/parque"
               }
            }
         ],
         "minimum_number_should_match": 1
      }
   }
}"""
docs = search_es(body)

Got 1720 documents


In [383]:
(begin_space, end_space, mistyped, normal, limbo) = get_ids_to_fix(docs, ["centroide de reserva/parque"], "centroide de uc")
to_fix = begin_space + end_space + mistyped + limbo

Analisando doc 0
Analisando doc 100
Analisando doc 200
Analisando doc 300
Analisando doc 400
Analisando doc 500
Analisando doc 600
Analisando doc 700
Analisando doc 800
Analisando doc 900
Analisando doc 1000
Analisando doc 1100
Analisando doc 1200
Analisando doc 1300
Analisando doc 1400
Analisando doc 1500
Analisando doc 1600
Analisando doc 1700



Foram encontrados 0 registros com espaço no inicio.
Foram encontrados 0 registros com espaço no final.
Foram encontrados 0 registros escritos errados.
Foram encontrados 1720 registros normais.
Foram encontrados 0 registros no limbo.


In [322]:
not_found = update_couch(to_fix, "centroide de uc")
print "Não foram encontrados %d documentos" % len(not_found)

Não foram encontrados 0 documentos
